In [ ]:
import cv2
import numpy as np
from keras.models import load_model

model = load_model('CNN_model.h5')

In [ ]:
#HAND DETECTION
#load the Haar cascade hand detection model
hand_cascade = cv2.CascadeClassifier('hand.xml')

def preprocess_image(image):
    resized_image = cv2.resize(image, (28, 28))
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    normalized_image = grayscale_image / 255.0
    reshaped_image = normalized_image.reshape(1, 28, 28, 1)
    return reshaped_image

cap = cv2.VideoCapture(0, cv2.CAP_AVFOUNDATION)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)  

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame")
        break
    
    #convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #detect hands in the frame
    hands = hand_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    #draw rectangles around the detected hands
    for (x, y, w, h) in hands:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        #adjust the region of interest (ROI) to focus on the hand
        roi = frame[y:y+h, x:x+w]
        
        #resize the ROI to the desired input size for the model
        resized_roi = cv2.resize(roi, (28, 28))
        
        #preprocess the resized ROI
        processed_frame = preprocess_image(resized_roi)
        
        #make prediction using the model
        prediction = model.predict(processed_frame)
        predicted_letter = chr(prediction.argmax() + 65)
        
        #display prediction
        cv2.putText(frame, predicted_letter, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow('ASL Prediction', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()